# xDB Training

A notebook exploring the xDB dataset tier3 part for Building Segmentation Model and Damaged Classification Model

In [1]:
# In a Jupyter notebook or IPython environment, run this in the first cell
%load_ext autoreload
%autoreload 2

In [11]:
%reload_ext autoreload

In [2]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

In [3]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
# Define the transformation pipeline
def get_train_augmentation_pipeline(image_size=(256, 256)):
    transform = A.Compose([
        # Resize images and masks
        A.Resize(image_size[0], image_size[1], p=1.0),  # Ensure both image and mask are resized
        # Random horizontal flip
        A.HorizontalFlip(p=0.5),
        # Random vertical flip
        A.VerticalFlip(p=0.5),
        # Random rotation
        A.RandomRotate90(p=0.5),
        # Random brightness and contrast adjustments
        A.RandomBrightnessContrast(p=0.2),
        # Random contrast adjustment
        A.RandomGamma(p=0.2),
        # Random scale and aspect ratio change
        A.RandomSizedCrop(min_max_height=(image_size[0], image_size[1]), height=image_size[0], width=image_size[1], p=0.5),
        # Random blur
        A.GaussianBlur(p=0.2),
        # Random crop (if you want to focus on specific regions)
        A.CenterCrop(height=image_size[0], width=image_size[1], p=1.0),
        # Convert to tensor (works for both image and mask)
        ToTensorV2()
    ])
    return transform

def get_val_augmentation_pipeline(image_size=(256, 256)):
    transform = A.Compose([
        # Resize images and masks
        A.Resize(image_size[0], image_size[1], p=1.0),  # Ensure both image and mask are resized
        ToTensorV2()
    ])
    return transform

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from datasets import xDB_Damaged_Building

origin_dir = "../data/xDB/tier3"

data_train  = xDB_Damaged_Building(
    origin_dir = origin_dir,
    mode="building",
    time="pre",
    transform=get_train_augmentation_pipeline(image_size=(512,512)),
    type="train",
    val_ratio=0.1, 
    test_ratio=0.1,
)

data_val  = xDB_Damaged_Building(
    origin_dir = origin_dir,
    mode="building",
    time="pre",
    transform=get_val_augmentation_pipeline(image_size=(512,512)),
    type="val",
    val_ratio=0.1, 
    test_ratio=0.1,
)

Loaded 5096 train labels.
Loaded 637 val labels.


In [6]:
from models import ResNet_UNET 

model = ResNet_UNET(
        in_channels=3,
        out_channels=2,
        backbone_name="resnet34",
        pretrained=True,
        freeze_backbone=False,
    )

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /home/onyxia/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 109MB/s] 


In [7]:
import torch
# Before training, wrap the model for Data Parallelism
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs with DataParallel")
    model = torch.nn.DataParallel(model)

model.to("cuda")


ResNet_UNET(
  (layer0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (layer1): Sequential(
    (0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [8]:
from torch.utils.data import DataLoader
train_dl = DataLoader(data_train, batch_size=32, shuffle=True,num_workers=8, pin_memory=True)
val_dl = DataLoader(data_val, batch_size=32, shuffle=False, num_workers=8, pin_memory=True)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from training import train 
from losses import SoftCrossEntropyLoss
from metrics import accuracy, f1_score, iou_score


optimizer = optim.Adam(params=filter(lambda p: p.requires_grad, model.parameters()), lr=1e-3)
params_opt = {}
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
params_sc = {}
criterion = nn.CrossEntropyLoss(weight=torch.tensor([0.4, 0.6]).to("cuda"))
# Define Metrics 
metrics = [accuracy, f1_score, iou_score]

# Early Stopping 
early_stopping_params = {"patience":5, "trigger_times": 0}

train(
    model,
    train_dl=train_dl,
    valid_dl=val_dl,
    optimizer=optimizer,
    scheduler=scheduler,
    params_opt=params_opt,
    params_sc=params_sc,
    metrics=metrics,
    nb_epochs=50,
    loss_fn=criterion,
    experiment_name="xDB_UNet",
    log_dir="../runs",
    model_dir="../models",
    early_stopping_params = early_stopping_params,
    image_key="image",
    training_log_interval=1,
    verbose=False
)

Epoch 1:  54%|█████▍    | 86/160 [01:52<01:33,  1.26s/batch, Loss=0.488]